In [1]:
import pandas as pd
import os
import itertools
from tqdm import tqdm
import joblib 
import json
import warnings

warnings.simplefilter("ignore", category=UserWarning)

In [8]:
merge_directory = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/output_phones/"
# output_directory = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/dataset/"
output_dir = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/output_jsonl_files/"

In [3]:
kmeans50 = joblib.load("/shared/3/projects/bangzhao/prosodic_embeddings/merge/clustering/kmeans_10000epi_50clu.pkl")
kmeans100 = joblib.load("/shared/3/projects/bangzhao/prosodic_embeddings/merge/clustering/kmeans_10000epi_100clu.pkl")
kmeans200 = joblib.load("/shared/3/projects/bangzhao/prosodic_embeddings/merge/clustering/kmeans_10000epi_200clu.pkl")
kmeans500 = joblib.load("/shared/3/projects/bangzhao/prosodic_embeddings/merge/clustering/kmeans_10000epi_500clu.pkl")
kmeans1000 = joblib.load("/shared/3/projects/bangzhao/prosodic_embeddings/merge/clustering/kmeans_10000epi_1000clu.pkl")
print('done.')

done.


In [4]:
files = [f for f in os.listdir(merge_directory) if os.path.isfile(os.path.join(merge_directory, f))]
print('done.')

done.


In [9]:
BATCH_SIZE = 500      # Write to file every 500 processed files
MAX_LINES = 50000     # Start a new file after 50,000 lines

buffer = []           # Temporary storage for records
total_lines = 0       # Track total lines written
file_count = 1        # Track number of JSONL files created

# Create initial output JSONL file
output_jsonl = os.path.join(output_dir, f"output_part_{file_count}.jsonl")

# Open the first file for writing
jsonl_file = open(output_jsonl, "w", encoding="utf-8")

In [ ]:
# Process files
for idx, filename in enumerate(tqdm(files, desc="Processing files", unit="file")):
    file_path = os.path.join(merge_directory, filename)

    try:
        # Read CSV file
        data = pd.read_csv(file_path)
        X = data[['F0semitoneFrom27.5Hz_sma3nz', 'F1frequency_sma3nz', 'mfcc1_sma3', 'mfcc2_sma3', 
                  'mfcc3_sma3', 'mfcc4_sma3', 'F0semitoneFrom27.5Hz_sma3nzSlope', 'F1frequency_sma3nzSlope',
                  'mfcc1_sma3Slope', 'mfcc2_sma3Slope', 'mfcc3_sma3Slope', 'mfcc4_sma3Slope']]

        # Apply clustering
        prosody_id_50 = kmeans50.predict(X)
        prosody_id_100 = kmeans100.predict(X)
        prosody_id_200 = kmeans200.predict(X)
        prosody_id_500 = kmeans500.predict(X)
        prosody_id_1000 = kmeans1000.predict(X)

        # Extract phoneme column
        phonemes = data['content']

        # Create DataFrame for result
        df_result = pd.DataFrame({
            'phoneme': phonemes, 
            'prosody_id_50': prosody_id_50,
            'prosody_id_100': prosody_id_100,
            'prosody_id_200': prosody_id_200,
            'prosody_id_500': prosody_id_500,
            'prosody_id_1000': prosody_id_1000,
        })

        # Convert DataFrame to JSON format
        json_record = {
            "name": filename,  # Store filename
            **df_result.to_dict(orient="list")  # Convert all columns to lists
        }

        # Add record to buffer
        buffer.append(json.dumps(json_record))
        total_lines += 1

        # Write buffer to file when it reaches BATCH_SIZE
        if len(buffer) >= BATCH_SIZE:
            jsonl_file.write("\n".join(buffer) + "\n")
            buffer = []  # Clear buffer

        # If total_lines reaches MAX_LINES, start a new file
        if total_lines >= MAX_LINES:
            jsonl_file.close()  # Close current file
            file_count += 1      # Increment file counter
            total_lines = 0      # Reset line counter

            # Create new JSONL file
            output_jsonl = os.path.join(output_dir, f"output_part_{file_count}.jsonl")
            jsonl_file = open(output_jsonl, "w", encoding="utf-8")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Write any remaining records in buffer
if buffer:
    jsonl_file.write("\n".join(buffer) + "\n")

# Close the last file
jsonl_file.close()

print(f"Processed {len(files)} files. JSONL files saved in {output_dir}.")

Processing files:   0%|                  | 710/444630 [00:55<5:11:56, 23.72file/s]

Error processing 739-httpsanchor.fms1f999408podcastplay14475003https3A2F2Fd3ctxlq1ktw2nl.cloudfront.net2Fproduction2F20204292F778317963200018b95348742b99.m4a.csv: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


Processing files:   0%|                 | 1182/444630 [01:30<5:22:05, 22.95file/s]

Error processing 1267-httpsmcdn.podbean.commfweb4janth031_our_undoing_radio_crazy_for_covid.mp3.csv: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


Processing files:   1%|                 | 2908/444630 [03:56<4:27:40, 27.50file/s]

...

Processing files:  11%|█▌            | 49350/444630 [2:06:48<6:10:42, 17.77file/s]

Error processing 50440-httpsanchor.fms13934f50podcastplay14853720https3A2F2Fd3ctxlq1ktw2nl.cloudfront.net2Fstaging2F2020572F80213726441002b44cc415c5e79.m4a.csv: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


Processing files:  12%|█▌           | 51386/444630 [2:10:59<10:50:48, 10.07file/s]

Error processing 52471-httpsapi.spreaker.comdownloadepisode29581658kogo_felderhoff_6_5_20.mp3.csv: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


Processing files:  12%|█▋            | 51978/444630 [2:11:58<6:37:16, 16.47file/s]